### Generation configs

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# === Config ===
model_name = "mistralai/Mistral-7B-v0.1"
checkpoint_path = "output/checkpoint-42600"

# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# === QLoRA quantization config ===
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# === Load model from fine-tuned checkpoint ===
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.eval()


### Prompt + generate

In [ ]:
# === Change the Input to generate
prompt = """### Instruction:
Write an academic paragraph given the title.

### Input: 
SnapMix

### Response:
"""

# === Tokenize and generate ===
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.95,
        do_sample=True
    )

# === Decode ===
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction:
Write an academic paragraph given the title.

### Input:
SnapMix

### Response:
This paper presents SnapMix, a new method to generate images from a text prompt. It leverages diffusion models by generating multiple images that are later combined. This approach allows for a more controlled and flexible image generation process compared to conventional diffusion models. The method is evaluated on the LLAMA-7B model, demonstrating improvements in image quality and diversity. Additionally, the paper introduces a new dataset, SnapMix-1M, which consists of 1 million text-image pairs generated by SnapMix. This dataset is designed to support further research and development in image generation. The code and dataset will be made publicly available. The code and dataset will be made publicly available at https://github.com/sakuraz
